In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import mglearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from pandas import ExcelWriter
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
#from openpyxl import Workbook

In [2]:
Twitter_Absolute_Sigma = pd.read_table('Twitter-Absolute-Sigma-500.data',sep=",")

In [3]:
Twitter_Absolute_Sigma.columns= ["NCD_0", "NCD_1", "NCD_2", "NCD_3", "NCD_4", "NCD_5", "NCD_6", "AI_0", "AI_1", "AI_2", "AI_3", "AI_4", "AI_5", "AI_6", "AS_NA_0", "AS_NA_1", "AS_NA_2", "AS_NA_3", "AS_NA_4", "AS_NA_5", "AS_NA_6", "BL_0", "BL_1", "BL_2", "BL_3", "BL_4", "BL_5", "BL_6", "NAC_0", "NAC_1", "NAC_2", "NAC_3", "NAC_4", "NAC_5", "NAC_6", "AS_NAC_0", "AS_NAC_1", "AS_NAC_2", "AS_NAC_3", "AS_NAC_4", "AS_NAC_5", "AS_NAC_6", "CS_0", "CS_1", "CS_2", "CS_3", "CS_4", "CS_5", "CS_6", "AT_0", "AT_1", "AT_2", "AT_3", "AT_4", "AT_5", "AT_6", "NA_0", "NA_1", "NA_2", "NA_3", "NA_4", "NA_5", "NA_6", "ADL_0", "ADL_1", "ADL_2", "ADL_3", "ADL_4", "ADL_5", "ADL_6", "NAD_0", "NAD_1", "NAD_2", "NAD_3", "NAD_4", "NAD_5", "NAD_6", "buzz"]

In [4]:
Twitter_Absolute_Sigma['buzz'] = Twitter_Absolute_Sigma['buzz'].map( {1.: 1, 0.: 0} ).astype(int)


In [5]:
#Twitter_Absolute_Sigma = Twitter_Absolute_Sigma[:5000]
X = Twitter_Absolute_Sigma

In [6]:
X= X.drop(['buzz'],axis = 1)
y = Twitter_Absolute_Sigma['buzz']

In [7]:
Dummytrain,sample_data,Dummytrain2,sample_target = train_test_split(X, y, shuffle = True, test_size = 0.1)


In [8]:
X = sample_data
y = sample_target

In [10]:
X.shape

(14071, 77)

In [9]:
X_train_org, X_test_org, y_train, y_test = train_test_split(X, y, random_state=0)

In [11]:
#We choose the MinMaxScaler to scale the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_org)
X_test = scaler.transform(X_test_org)

X_full_scaled = scaler.fit_transform(X)

# Apply four voting classifiers - two with hard voting and two with soft voting

In [ ]:
log_clf = LogisticRegression(C=100)
log_clf.fit(X_train, y_train)
knn_clf = KNeighborsClassifier(20)
knn_clf.fit(X_train, y_train)

dtc =DecisionTreeClassifier(max_depth=3)
dtc.fit(X_train, y_train)
svc_lin = LinearSVC(C=10)
svc_lin.fit(X_train, y_train)

LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [ ]:

voting_clf = VotingClassifier(estimators=[('dtc', dtc),('svc_lin',svc_lin)], voting='hard')
voting_clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
for clf in (dtc, svc_lin, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))


DecisionTreeClassifier 0.9593519044911881
LinearSVC 0.9633314383172257
VotingClassifier 0.9624786810687891


In [125]:
voting_clf_soft = VotingClassifier(estimators=[('lr', log_clf), ('knn', knn_clf)], voting='soft')
voting_clf_soft.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
for clf in (log_clf, knn_clf, voting_clf_soft):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9627629334849346
KNeighborsClassifier 0.9602046617396248
VotingClassifier 0.9633314383172257


# Apply any two models with bagging and any two models with pasting

##### KNeighborsClassifier with Bagging

In [28]:
from sklearn.ensemble import BaggingClassifier

#From Project 2 we have seen that the optimum value of n_neighbors is 20
model_name_knn = 'KNeighborsClassifier(Bagging)'
knn = KNeighborsClassifier(n_neighbors=20)
bag_knn = BaggingClassifier(knn, n_estimators = 100, max_samples= 500, n_jobs = -1, random_state=0)
bag_knn.fit(X_train, y_train)
train_score = bag_knn.score(X_train, y_train)
test_score = bag_knn.score(X_test, y_test)

Parameters = 'n_neighbors=20'

In [29]:
print('KNeighborsClassifier Bagging: train_score',train_score)
print('KNeighborsClassifier Bagging: test_score',test_score)

KNeighborsClassifier Bagging: train_score 0.9546100634890552
KNeighborsClassifier Bagging: test_score 0.9565093803297328


In [30]:
report_table = []

In [31]:
report_table = [[model_name_knn, Parameters, train_score, test_score]]

report_tabledf = pd.DataFrame(report_table, columns = ['Model', 'Parameters from Proj2', 'Train accuracy', 'Test accuracy']).set_index('Model', drop=True)
report_tabledf

,Parameters from Proj2,Train accuracy,Test accuracy
Model,,,
KNeighborsClassifier(Bagging),n_neighbors=20,0.95461,0.956509


##### LogisticRegression with bagging

In [32]:
#LogisticRegression with bagging
model_name_logreg = 'LogisticRegression with bagging'
logreg = LogisticRegression(C=100)

In [33]:
bag_lr = BaggingClassifier(logreg, n_estimators = 100, max_samples= 500, n_jobs = -1, random_state=0)
bag_lr.fit(X_train, y_train)
train_score = bag_lr.score(X_train, y_train)
test_score = bag_lr.score(X_test, y_test)

Parameters = 'C=100'

In [34]:
print('logreg Bagging: train_score',train_score)
print('logreg Bagging: test_score',test_score)

logreg Bagging: train_score 0.9602008907419691
logreg Bagging: test_score 0.9602046617396248


In [36]:
report_table = report_table + [[model_name_logreg, Parameters, train_score, test_score]]

report_tabledf = pd.DataFrame(report_table, columns = ['Model', 'Parameters from Proj2', 'Train accuracy', 'Test accuracy']).set_index('Model', drop=True)
report_tabledf

,Parameters from Proj2,Train accuracy,Test accuracy
Model,,,
KNeighborsClassifier(Bagging),n_neighbors=20,0.954610,0.956509
LogisticRegression with bagging,C=100,0.960201,0.960205


##### Linear SVC with Pasting

In [35]:
model_name_lin = 'LinearSVC with pasting'

svc_lin = LinearSVC(C=10)

In [37]:
bag_lin = BaggingClassifier(svc_lin, n_estimators = 100, max_samples= 500,bootstrap=False, n_jobs = -1, random_state=0)
bag_lin.fit(X_train, y_train)
train_score_lin = bag_lin.score(X_train, y_train)
test_score_lin = bag_lin.score(X_test, y_test)

Parameters = 'C=10'

In [38]:
print('LinearSVC with pasting: train_score',train_score_lin)
print('LinearSVC with pasting: test_score',test_score_lin)

LinearSVC with pasting: train_score 0.9655074386430399
LinearSVC with pasting: test_score 0.9613416714042069


In [39]:
report_table = report_table + [[model_name_lin, Parameters, train_score, test_score]]

report_tabledf = pd.DataFrame(report_table, columns = ['Model', 'Parameters from Proj2', 'Train accuracy', 'Test accuracy']).set_index('Model', drop=True)
report_tabledf

,Parameters from Proj2,Train accuracy,Test accuracy
Model,,,
KNeighborsClassifier(Bagging),n_neighbors=20,0.954610,0.956509
LogisticRegression with bagging,C=100,0.960201,0.960205
LinearSVC with pasting,C=10,0.960201,0.960205


In [40]:
model_name_dtc = 'DTree Classifier with pasting'

dtc =DecisionTreeClassifier(max_depth=3)

bag_dtc = BaggingClassifier(dtc, n_estimators = 100, max_samples= 500,bootstrap=False, n_jobs = -1, random_state=0)
bag_dtc.fit(X_train, y_train)
train_score_dtc = bag_dtc.score(X_train, y_train)
test_score_dtc = bag_dtc.score(X_test, y_test)

Parameters = 'max_depth=3'

print('DTree Classifier with pasting: train_score',train_score_dtc)
print('DTree Classifier with pasting: test_score',test_score_dtc)


DTree Classifier with pasting: train_score 0.9683502321614706
DTree Classifier with pasting: test_score 0.9641841955656623


In [41]:
report_table = report_table + [[model_name_dtc, Parameters, train_score, test_score]]

report_tabledf = pd.DataFrame(report_table, columns = ['Model', 'Parameters from Proj2', 'Train accuracy', 'Test accuracy']).set_index('Model', drop=True)
report_tabledf

,Parameters from Proj2,Train accuracy,Test accuracy
Model,,,
KNeighborsClassifier(Bagging),n_neighbors=20,0.954610,0.956509
LogisticRegression with bagging,C=100,0.960201,0.960205
LinearSVC with pasting,C=10,0.960201,0.960205
DTree Classifier with pasting,max_depth=3,0.960201,0.960205


# Apply any two models with adaboost boosting

In [43]:
model_name_ada_dtc = 'DTree Classifier (Adaboost)'

dtc =DecisionTreeClassifier(max_depth=3)
adatree_dtc = AdaBoostClassifier(dtc, n_estimators = 100, learning_rate= 0.5, random_state=0)

In [44]:
adatree_dtc.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.5, n_estimators=100, random_state=0)

In [45]:
train_score = adatree_dtc.score(X_train, y_train)
test_score = adatree_dtc.score(X_test, y_test)
print('DTree Classifier (Adaboost): train_score',train_score_dtc)
print('DTree Classifier (Adaboost): test_score',test_score_dtc)

DTree Classifier (Adaboost): train_score 0.9683502321614706
DTree Classifier (Adaboost): test_score 0.9641841955656623


In [46]:
report_table = report_table + [[model_name_ada_dtc, Parameters, train_score, test_score]]

report_tabledf = pd.DataFrame(report_table, columns = ['Model', 'Parameters from Proj2', 'Train accuracy', 'Test accuracy']).set_index('Model', drop=True)
report_tabledf

,Parameters from Proj2,Train accuracy,Test accuracy
Model,,,
KNeighborsClassifier(Bagging),n_neighbors=20,0.954610,0.956509
LogisticRegression with bagging,C=100,0.960201,0.960205
LinearSVC with pasting,C=10,0.960201,0.960205
DTree Classifier with pasting,max_depth=3,0.960201,0.960205
DTree Classifier (Adaboost),max_depth=3,1.000000,0.963616


##### Kernel SVC with Adaboost

In [55]:
model_nameLogisticRegression = 'LogisticRegression (Adaboost)'

logreg = LogisticRegression(C=100)
#When nothing is passed to the Kernel parameter by default it takes 'rbf'
adatree_logreg = AdaBoostClassifier(logreg, n_estimators = 100, learning_rate= 0.5, random_state=0)


In [56]:
adatree_logreg.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          learning_rate=0.5, n_estimators=100, random_state=0)

In [57]:
train_score = adatree_logreg.score(X_train, y_train)
test_score = adatree_logreg.score(X_test, y_test)
print('LogisticRegression (Adaboost): train_score',train_score_dtc)
print('LogisticRegression (Adaboost): test_score',test_score_dtc)

LogisticRegression (Adaboost): train_score 0.9683502321614706
LogisticRegression (Adaboost): test_score 0.9641841955656623


In [ ]:
report_table = report_table + [[model_nameLogisticRegression, Parameters, train_score, test_score]]

report_tabledf = pd.DataFrame(report_table, columns = ['Model', 'Parameters from Proj2', 'Train accuracy', 'Test accuracy']).set_index('Model', drop=True)
report_tabledf

# Apply one model with gradient boosting

In [60]:
from sklearn.ensemble import GradientBoostingClassifier

In [61]:
model_name_gbrt ='GradientBoostingClassifier'

gbrt = GradientBoostingClassifier(max_depth=3, random_state=0)
param_grid = {'n_estimators': [50, 100, 200], 'learning_rate':[0.1, 0.5, 1.0]}

In [62]:
grid_gbrt = GridSearchCV(gbrt, param_grid, cv=5, n_jobs=-1, return_train_score=True)

In [63]:
grid_gbrt.fit(X_train, y_train)
train_score = grid_gbrt.score(X_train, y_train)
test_score = grid_gbrt.score(X_test, y_test)

In [64]:
best_parameters = str(grid_gbrt.best_params_)
print("Best parameters: {}".format(grid_gbrt.best_params_))

Best parameters: {'learning_rate': 0.1, 'n_estimators': 50}


In [65]:
report_table = report_table + [[model_name_gbrt, Parameters, train_score, test_score]]

report_tabledf = pd.DataFrame(report_table, columns = ['Model', 'Parameters from Proj2', 'Train accuracy', 'Test accuracy']).set_index('Model', drop=True)
report_tabledf

,Parameters from Proj2,Train accuracy,Test accuracy
Model,,,
KNeighborsClassifier(Bagging),n_neighbors=20,0.954610,0.956509
LogisticRegression with bagging,C=100,0.960201,0.960205
LinearSVC with pasting,C=10,0.960201,0.960205
DTree Classifier with pasting,max_depth=3,0.960201,0.960205
DTree Classifier (Adaboost),max_depth=3,1.000000,0.963616
LogisticRegression (Adaboost),max_depth=3,0.947977,0.946276
GradientBoostingClassifier,max_depth=3,0.977352,0.965890


# PCA

Apply PCA on data and then apply all the models in project 2 again on data you get from PCA. Compare your results with results in project 2. You don't need to apply all the models twice. Just copy the result table from project 2, prepare similar table for all the models after PCA and compare both tables. Does PCA help in getting better results?

In [68]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std_scaled = scaler.fit_transform(X)
pca = PCA(n_components = 0.95)
X_reduced = pca.fit_transform(X_std_scaled)


In [69]:
X_reduced.shape

(14071, 15)

In [70]:
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_reduced, y, random_state=0)

In [71]:
X_train_pca.shape

(10553, 15)

In [72]:
X_test_pca.shape

(3518, 15)

In [106]:
model_name = 'KNeighborsClassifier with PCA'
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_train_pca, y_train)
train_score = knn.score(X_train_pca, y_train)
test_score = knn.score(X_test_pca, y_test)
print("Train set accuracy: {:.2f}".format(knn.score(X_train_pca, y_train)))
print("Test set accuracy: {:.2f}".format(knn.score(X_test_pca, y_test)))
y_knn_predict = knn.predict(X_test_pca)
y_knn_train_predict = knn.predict(X_train_pca)
from sklearn.metrics import roc_auc_score

auc_train_score = roc_auc_score(y_knn_train_predict, y_train)
auc_test_score = roc_auc_score(y_knn_predict, y_test)

print('Train roc_auc_score: %.2f'%roc_auc_score(y_knn_train_predict, y_train))
print('Test roc_auc_score: %.2f '%roc_auc_score(y_knn_predict, y_test))

Train set accuracy: 0.97
Test set accuracy: 0.96
Train roc_auc_score: 0.96
Test roc_auc_score: 0.95 


In [107]:
report_PCA = []

In [108]:
report_PCA = [[model_name, "n_neighbors=20" , train_score, test_score, auc_train_score, auc_test_score]]

In [109]:
report_table_PCA = pd.DataFrame(report_PCA, columns = ['Model', 'Parameters', 'Train accuracy', 'Test accuracy', 'Train AUC Score', 'Test AUC Score']).set_index('Model', drop=True)

In [110]:
report_table_PCA

,Parameters,Train accuracy,Test accuracy,Train AUC Score,Test AUC Score
Model,,,,,
KNeighborsClassifier with PCA,n_neighbors=20,0.966455,0.963616,0.956417,0.952917


In [111]:
#LogisticRegression with PCA

In [112]:
model_name = 'LogisticRegression with PCA'
logreg_PCA = LogisticRegression(C=100)
logreg_PCA.fit(X_train_pca, y_train)
train_score = logreg_PCA.score(X_train_pca, y_train)
test_score = logreg_PCA.score(X_test_pca, y_test)
print("Train set accuracy: {:.2f}".format(logreg_PCA.score(X_train_pca, y_train)))
print("Test set accuracy: {:.2f}".format(logreg_PCA.score(X_test_pca, y_test)))
y_logreg_predict = logreg_PCA.predict(X_test_pca)
y_logreg_train_predict = logreg_PCA.predict(X_train_pca)
from sklearn.metrics import roc_auc_score

auc_train_score = roc_auc_score(y_logreg_train_predict, y_train)
auc_test_score = roc_auc_score(y_logreg_predict, y_test)

print('Train roc_auc_score: %.2f'%roc_auc_score(y_logreg_train_predict, y_train))
print('Test roc_auc_score: %.2f '%roc_auc_score(y_logreg_predict, y_test))

Train set accuracy: 0.97
Test set accuracy: 0.96
Train roc_auc_score: 0.96
Test roc_auc_score: 0.95 


In [113]:
report_PCA = report_PCA + [[model_name, "C=100" , train_score, test_score, auc_train_score, auc_test_score]]
report_table_PCA = pd.DataFrame(report_PCA, columns = ['Model', 'Parameters', 'Train accuracy', 'Test accuracy', 'Train AUC Score', 'Test AUC Score']).set_index('Model', drop=True)
report_table_PCA

,Parameters,Train accuracy,Test accuracy,Train AUC Score,Test AUC Score
Model,,,,,
KNeighborsClassifier with PCA,n_neighbors=20,0.966455,0.963616,0.956417,0.952917
LogisticRegression with PCA,C=100,0.965507,0.962763,0.955021,0.951711


In [114]:
#'LinearSVC with PCA'
model_name = 'LinearSVC with PCA'
svc_lin_PCA = LinearSVC(C=1)
svc_lin_PCA.fit(X_train_pca, y_train)
train_score = svc_lin_PCA.score(X_train_pca, y_train)
test_score = svc_lin_PCA.score(X_test_pca, y_test)
print("Train set accuracy: {:.2f}".format(svc_lin_PCA.score(X_train_pca, y_train)))
print("Test set accuracy: {:.2f}".format(svc_lin_PCA.score(X_test_pca, y_test)))
y_lin_predict = svc_lin_PCA.predict(X_test_pca)
y_lin_train_predict = svc_lin_PCA.predict(X_train_pca)
from sklearn.metrics import roc_auc_score

auc_train_score = roc_auc_score(y_lin_train_predict, y_train)
auc_test_score = roc_auc_score(y_lin_predict, y_test)

print('Train roc_auc_score: %.2f'%roc_auc_score(y_lin_train_predict, y_train))
print('Test roc_auc_score: %.2f '%roc_auc_score(y_lin_predict, y_test))

Train set accuracy: 0.96
Test set accuracy: 0.96
Train roc_auc_score: 0.96
Test roc_auc_score: 0.95 


In [115]:
report_PCA = report_PCA + [[model_name, "C=1" , train_score, test_score, auc_train_score, auc_test_score]]
report_table_PCA = pd.DataFrame(report_PCA, columns = ['Model', 'Parameters', 'Train accuracy', 'Test accuracy', 'Train AUC Score', 'Test AUC Score']).set_index('Model', drop=True)
report_table_PCA

,Parameters,Train accuracy,Test accuracy,Train AUC Score,Test AUC Score
Model,,,,,
KNeighborsClassifier with PCA,n_neighbors=20,0.966455,0.963616,0.956417,0.952917
LogisticRegression with PCA,C=100,0.965507,0.962763,0.955021,0.951711
LinearSVC with PCA,C=1,0.964086,0.961910,0.956903,0.953870


In [116]:
#'Kernel SVM with PCA'
model_name = 'Kernel SVM with PCA'
SVC_PCA = SVC(C=100,gamma=0.1)
SVC_PCA.fit(X_train_pca, y_train)
train_score = SVC_PCA.score(X_train_pca, y_train)
test_score = SVC_PCA.score(X_test_pca, y_test)
print("Train set accuracy: {:.2f}".format(SVC_PCA.score(X_train_pca, y_train)))
print("Test set accuracy: {:.2f}".format(SVC_PCA.score(X_test_pca, y_test)))
y_SVC_predict = SVC_PCA.predict(X_test_pca)
y_SVC_train_predict = SVC_PCA.predict(X_train_pca)
from sklearn.metrics import roc_auc_score

auc_train_score = roc_auc_score(y_SVC_train_predict, y_train)
auc_test_score = roc_auc_score(y_SVC_predict, y_test)

print('Train roc_auc_score: %.2f'%roc_auc_score(y_SVC_train_predict, y_train))
print('Test roc_auc_score: %.2f '%roc_auc_score(y_SVC_predict, y_test))

Train set accuracy: 0.97
Test set accuracy: 0.96
Train roc_auc_score: 0.96
Test roc_auc_score: 0.94 


In [117]:
report_PCA = report_PCA + [[model_name, "C=100,gamma=0.1" , train_score, test_score, auc_train_score, auc_test_score]]
report_table_PCA = pd.DataFrame(report_PCA, columns = ['Model', 'Parameters', 'Train accuracy', 'Test accuracy', 'Train AUC Score', 'Test AUC Score']).set_index('Model', drop=True)
report_table_PCA

,Parameters,Train accuracy,Test accuracy,Train AUC Score,Test AUC Score
Model,,,,,
KNeighborsClassifier with PCA,n_neighbors=20,0.966455,0.963616,0.956417,0.952917
LogisticRegression with PCA,C=100,0.965507,0.962763,0.955021,0.951711
LinearSVC with PCA,C=1,0.964086,0.961910,0.956903,0.953870
Kernel SVM with PCA,"C=100,gamma=0.1",0.973278,0.958783,0.964722,0.941199


In [118]:

#'Decision Tree with PCA'
model_name = 'Decision Tree with PCA'
dtc_PCA =DecisionTreeClassifier(max_depth=3)
dtc_PCA.fit(X_train_pca, y_train)
train_score = dtc_PCA.score(X_train_pca, y_train)
test_score = dtc_PCA.score(X_test_pca, y_test)
print("Train set accuracy: {:.2f}".format(dtc_PCA.score(X_train_pca, y_train)))
print("Test set accuracy: {:.2f}".format(dtc_PCA.score(X_test_pca, y_test)))
y_dtc_train_predict = dtc_PCA.predict(X_train_pca)
y_dtc_predict = dtc_PCA.predict(X_test_pca)
from sklearn.metrics import roc_auc_score

auc_train_score = roc_auc_score(y_dtc_train_predict, y_train)
auc_test_score = roc_auc_score(y_dtc_predict, y_test)

print('Train roc_auc_score: %.2f'%roc_auc_score(y_dtc_train_predict, y_train))
print('Test roc_auc_score: %.2f '%roc_auc_score(y_dtc_predict, y_test))

Train set accuracy: 0.96
Test set accuracy: 0.96
Train roc_auc_score: 0.94
Test roc_auc_score: 0.93 


In [119]:
report_PCA = report_PCA + [[model_name, "max_depth=3" , train_score, test_score, auc_train_score, auc_test_score]]
report_table_PCA = pd.DataFrame(report_PCA, columns = ['Model', 'Parameters', 'Train accuracy', 'Test accuracy', 'Train AUC Score', 'Test AUC Score']).set_index('Model', drop=True)

report_table_PCA

,Parameters,Train accuracy,Test accuracy,Train AUC Score,Test AUC Score
Model,,,,,
KNeighborsClassifier with PCA,n_neighbors=20,0.966455,0.963616,0.956417,0.952917
LogisticRegression with PCA,C=100,0.965507,0.962763,0.955021,0.951711
LinearSVC with PCA,C=1,0.964086,0.961910,0.956903,0.953870
Kernel SVM with PCA,"C=100,gamma=0.1",0.973278,0.958783,0.964722,0.941199
Decision Tree with PCA,max_depth=3,0.963991,0.958499,0.938292,0.930586


In [121]:
writer = pd.ExcelWriter('Classification PCA Report final.xlsx')
report_tabledf.to_excel(writer,'PCA')
writer.save()

In [122]:
project2_report = pd.read_excel('C:/Users/Test/Downloads/ClassificationFinalReport.xlsx')

In [123]:
project2_report

,Model,Best Parameters,Train accuracy,Test accuracy,Train AUC Score,Test AUC Score
0,KNeighborsClassifier,{'n_neighbors': 20},0.992840,0.982360,0.955569,0.955244
1,LogisticRegression,{'C': 100},0.992184,0.991584,0.955776,0.949502
2,LinearSVC,{'C': 10},0.992256,0.991565,0.958279,0.955885
3,KernelSVC,"{'C': 100, 'gamma': 0.1}",0.991328,0.990294,0.956659,0.950439
4,DecisionTree,{'max_depth': 3},0.986085,0.980464,0.955950,0.941574


# Apply deep learning models covered in class

In [13]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [14]:
def create_model():
    #create model
    model = Sequential()
    model.add(Dense(45, input_dim=77, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [15]:
model = KerasClassifier(build_fn = create_model, verbose = 0)

In [16]:
param_grid = {'batch_size':[10,20,30,40] , 'epochs':[10, 50, 100]}
grid_search = GridSearchCV(estimator= model, param_grid = param_grid, cv = 5)

In [17]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001DD0FBDEBA8>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'batch_size': [10, 20, 30, 40], 'epochs': [10, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [19]:
print(grid_search.best_params_)

{'batch_size': 20, 'epochs': 50}


In [20]:
#Now Building the proper Deep Learning Model

In [24]:
bestmodel = Sequential()
bestmodel.add(Dense(45, input_dim=77, activation='relu'))
bestmodel.add(Dense(25, activation='relu'))
bestmodel.add(Dense(6, activation='relu'))
bestmodel.add(Dense(1, activation='sigmoid'))
    
#compile model
bestmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
# Fit the model
bestmodel.fit(X_train, y_train, epochs=50, batch_size=20)

Epoch 1/50
10553/10553 [==============================] - 4s 346us/step - loss: 0.2268 - acc: 0.9124
Epoch 2/50
10553/10553 [==============================] - 1s 134us/step - loss: 0.1090 - acc: 0.9585
Epoch 3/50
10553/10553 [==============================] - 1s 136us/step - loss: 0.1035 - acc: 0.9594
Epoch 4/50
10553/10553 [==============================] - 1s 134us/step - loss: 0.1019 - acc: 0.9611
Epoch 5/50
10553/10553 [==============================] - 1s 134us/step - loss: 0.0999 - acc: 0.9611
Epoch 6/50
10553/10553 [==============================] - 1s 138us/step - loss: 0.0999 - acc: 0.9619
Epoch 7/50
10553/10553 [==============================] - 2s 143us/step - loss: 0.0964 - acc: 0.9619
Epoch 8/50
10553/10553 [==============================] - 1s 142us/step - loss: 0.0945 - acc: 0.9622
Epoch 9/50
10553/10553 [==============================] - 1s 135us/step - loss: 0.0933 - acc: 0.9640
Epoch 10/50
10553/10553 [==============================] - 1s 137us/step - loss: 0.0932 - a

In [27]:
# evaluate the model
scores = bestmodel.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (bestmodel.metrics_names[1], scores[1]*100))

3518/3518 [==============================] - 0s 66us/step

acc: 96.84%


# Report

Project 3 is an extension of the findings of project 2. We have explored the dataset, there are no missing values and no categorical variables. For ensemble modelling, like all the individual regression models, we have used MinMaxScaler for scaling.

#####  Ensemble models
It is observed that Soft voting gives overall higher accuracy scores. LinearSVC with Hard voting has the highest accuracy score within the Hard voting classifier of 96.33% and Logistic Regression with Soft voting has the highest accuracy score within the Soft voting classifier of 96.13%.
Implemented KNeighborsClassifier and LogisticRegression with Bagging, LinearSVC and DTree Classifier with pasting & DTree Classifier LogisticRegression using Adaboost. GradientBoostingClassifier with max_depth=3 had good accuracy.
The best Ensemble model is Decision Tree classifier with Adaboost with 100% Train accuracy and 96.36% test accuracy.

#####  PCA
Standard scaler is recommended for PCA hence used that for scale the original X data and then applied PCA on it. Ran then PCA model with 0.95 variance and got 15 reduced components. Ran all the models from Projects 2 on the reduced data after PCA. After comparing these results with those of previous project, we observed that the accuracy of the train accuracy improved slightly after performing KNN on the PCA reduced dataset. Also observed approx 1% increase in the accuracy after running Kernel SVM on the PCA reduced dataset. However, the test accuracy has reduced for other models like Logistic regression, Decision tree and Kernalized SVM after using PCA reduced dataset. The reduction in accuracy may be attributed to the fact that the underlying sampling data might have changed from Project 2 to Project 3 that is the 10% sample of Project 2 might be slightly different from that selected in Project 3.
Best model after PCA is Kernal SVM.

#####  Deep learning
Implemented the The deep learning model having Objective Function/Loss Function of Binary cross-entropy
Ran grid search on KerasClassifier and found the best parameter is batch_size of 20 and epochs of 50. Built a model on this and got the accuracy of 96.84%